<a href="https://colab.research.google.com/github/praneshnikhar/Deep-Learning-models/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [2]:
print(tf.__version__)

2.18.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist


In [4]:
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
train_images = train_images/255.0

In [6]:
test_images = test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images= train_images.reshape(len(train_images),28,28,1)

In [11]:
test_images= test_images.reshape(len(test_images),28,28,1)

In [15]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape = (28,28,1)
    ),
    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')

])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model


In [21]:
!pip install keras-tuner

from keras_tuner import RandomSearch

from keras_tuner.engine.hyperparameters import HyperParameters


In [22]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 4 Complete [00h 23m 07s]
val_accuracy: 0.859000027179718

Best val_accuracy So Far: 0.9041666388511658
Total elapsed time: 01h 06m 52s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
112               |64                |conv_1_filter
3                 |5                 |conv_1_kernel
32                |48                |conv_2_filter
5                 |5                 |conv_2_kernel
64                |48                |dense_1_units
0.001             |0.001             |learning_rate

Epoch 1/3
1662/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 268ms/step - accuracy: 0.8007 - loss: 0.5519

In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()